In [1]:
# 3-layered neural network to learn the MNIST dataset

import numpy
import scipy.special # for sigmoid function
import matplotlib.pyplot
%matplotlib inline

In [2]:
class neuralNetwork:
    
    def __init__(self, inputNodes, hiddenNodes, outputNodes, learningRate):
        self.inodes = inputNodes
        self.hnodes = hiddenNodes
        self.onodes = outputNodes
        
        # linking weight matrices between input and hidden nodes, then hidden nodes and output nodes
        self.weightInHid = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.weightHidOut = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))
        
        # learning rate
        self.learnRate = learningRate
        
        # using sigmoid function as activation function
        self.activation_func = lambda x: scipy.special.expit(x)
        
        pass
    
    
    # training
    def train(self, inputs_list, targets_list):
        # turning input list into 2D array
        inputs = numpy.array(inputs_list, ndmin = 2).T
        targets = numpy.array(targets_list, ndmin = 2).T
    
        #signals into hidden layer
        hidden_inputs = numpy.dot(self.weightInHid, inputs)
        # signals from hidden layer
        hidden_outputs = self.activation_func(hidden_inputs)
    
        # signals into last output layer
        last_inputs = numpy.dot(self.weightHidOut, hidden_outputs)
        # signals from last output layer
        last_outputs = self.activation_func(last_inputs)
    
        # output error
        output_errors = targets - last_outputs
        # hidden layer error
        hidden_errors = numpy.dot(self.weightHidOut.T, output_errors)
        
        # update weights for links between hidden and output layers
        self.weightHidOut += self.learnRate * numpy.dot((output_errors * last_outputs * (1.0 - last_outputs)), numpy.transpose(hidden_outputs))
    
        # now for input and hidden layers
        self.weightInHid += self.learnRate * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
    
        pass
    
    # query neural network
    def query(self, inputs_list):
        # turn inputs list into 2d array
        inputs = numpy.array(inputs_list, ndmin = 2).T
    
        # signals into hidden
        hidden_inputs = numpy.dot(self.weightInHid, inputs)
        # signals from hidden
        hidden_outputs = self.activation_func(hidden_inputs)
    
        # signals into last layer
        last_inputs = numpy.dot(self.weightHidOut, hidden_outputs)
        # signals from last layer
        last_outputs = self.activation_func(last_inputs)
    
        return last_outputs

In [3]:
# number of nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
lr = 0.1

# creating instance of neural network
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, lr)

In [4]:
# load training CSV file
train_file = open("data/mnist_train.csv",'r')
train_list = train_file.readlines()
train_file.close()

In [5]:
# train our neural network

# epochs = number of times training data set was used
epochs = 5

for e in range(epochs):
    # go through every row in training set
    for record in train_list:
        all_values = record.split(',')
        # scale and shift inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create target output values
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass



In [6]:
test_file = open("data/mnist_test.csv",'r')
test_list = test_file.readlines()
test_file.close()

In [7]:
# test neural network

scores = []

for record in test_list:
    all_values = record.split(',')
    # correct answer is the first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # query network
    outputs = n.query(inputs)
    # index of highest value corresponds to the label
    label = numpy.argmax(outputs)
    # add right or wrong to list
    if (label == correct_label):
        scores.append(1)
    else:
        scores.append(0)
        pass
    pass


In [8]:
# calculate performance score
scores_array = numpy.asarray(scores)
print("performance = ", scores_array.sum() / scores_array.size)

performance =  0.9745
